In [104]:
import requests
import pandas as pd
from tqdm import tqdm 
from bs4 import BeautifulSoup
import os
import mimetypes
import numpy as np
import os

# selenium method
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


import time

# analysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


# duckdb
import duckdb
from collections import defaultdict


##### Class Download File

In [105]:
class download_file:
    def status_bar_api(self, api_url:str, csv_filename:str = None):
        # Stream the download
        response = requests.get(api_url, stream=True)
        response.raise_for_status()

        # Determine filename
        if "Content-Disposition" in response.headers:
            content_disposition = response.headers["Content-Disposition"]
            filename = content_disposition.split("filename=")[-1].strip('"')
        else:
            filename = os.path.basename(api_url)

        # If filename has no extension, try to guess from Content-Type
        if "." not in filename:
            content_type = response.headers.get("Content-Type", "")
            extension = mimetypes.guess_extension(content_type.split(";")[0].strip())
            if extension:
                filename += extension

        # Get total file size for progress bar (in bytes)
        total_size = int(response.headers.get("content-length", 0))
        chunk_size = 8192  # 8 KB per chunk

        # Download with progress bar
        with open(filename, "wb") as f, tqdm(
            total=total_size, unit='B', unit_scale=True, desc=filename
        ) as progress_bar:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    progress_bar.update(len(chunk))
    
    def wait_for_download(self, download_dir, timeout=60):
        """Wait until a new file appears in download_dir (crdownload removed)."""
        seconds = 0
        while seconds < timeout:
            files = os.listdir(download_dir)
            downloading = [f for f in files if f.endswith(".crdownload")]
            if not downloading and files:
                # return the most recently created file
                return os.path.join(download_dir, max(files, key=lambda f: os.path.getctime(os.path.join(download_dir, f))))
            time.sleep(1)
            seconds += 1
        raise TimeoutError("Download did not complete in time.")

    # download from cer 
    def download_from_cer(self, url:str):
        """Extract API URL from CER datasets using Selenium and download CSV."""
        download_dir = os.getcwd()

        # Setup Chrome in headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Configure downloads
        prefs = {
            "download.default_directory": download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        }
        chrome_options.add_experimental_option("prefs", prefs)

        # Start browser
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)

        wait = WebDriverWait(driver, 20)

        # Click the "Download CSV" button
        download_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Download CSV')]"))
        )
        download_button.click()

        # Wait for download to complete
        time.sleep(10)

        driver.quit()
        downloaded_file = self.wait_for_download(download_dir)

        print(f"Download Complete:{downloaded_file}")
        
    
    def download_cer_markets(self, url:str):
        url_header = url.split('/')[2]
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
            # Step 2: Find the button/link with the XLSX file
        div_tags = soup.find_all("div", class_="cer-accordion__body__item")
        for div in div_tags:
            a_tag = div.find("a", href=True)
            if a_tag:
                text = a_tag.get_text(strip=True).lower()  # normalize text
                if "csv" in text and ("power stations" in text and "projects" in text):
                    file_href = a_tag["href"]
                    full_url = f"https://www.{url_header}{file_href}"
                    self.status_bar_api(api_url=full_url,csv_filename=None)
    
    def download_abs(self, url:str):
        url_header = url.split('/')[2]
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find the link for "Population and people"
        target_div = None
        for div in soup.find_all("div", class_="file-description-link-formatter"):
            h4 = div.find("h4")
            if h4 and "Economy and industry" in h4.text:
                target_div = div
                break
        if target_div:
            a_tag = target_div.find("a", href=True)
            relative_url = a_tag['href']
            download_url = f"https://{url_header}" + relative_url
            print("Found download URL:", download_url)
            self.status_bar_api(api_url=download_url,csv_filename=None)


<h1>Retrieve Data Set </h1>

In [106]:

download = download_file()

# first dataset
print("Processing Download First Dataset ....")
cer_url = "https://data.cer.gov.au/datasets/NGER/ID0243"
download.download_from_cer(url=cer_url)
# second dataset
print("Processing Download Second Dataset ....")
cer_markets_url = "https://cer.gov.au/markets/reports-and-data/large-scale-renewable-energy-data"
download.download_cer_markets(url=cer_markets_url)
# third dataset
print("Processing Download Third Dataset ....")
abs_url = "https://www.abs.gov.au/methodologies/data-region-methodology/2011-24#data-downloads"
download.download_abs(url=abs_url)


Processing Download First Dataset ....
Download Complete:/Users/stephensusanto/Desktop/Intro to Programming/DE/NGER.ID0243.csv
Processing Download Second Dataset ....


power-stations-and-projects-accredited.csv: 100%|██████████| 25.8k/25.8k [00:00<00:00, 144kB/s]
power-stations-and-projects-committed.csv: 100%|██████████| 1.89k/1.89k [00:00<00:00, 3.45MB/s]
power-stations-and-projects-probable.csv: 100%|██████████| 2.22k/2.22k [00:00<00:00, 266kB/s]


Processing Download Third Dataset ....
Found download URL: https://www.abs.gov.au/methodologies/data-region-methodology/2011-24/14100DO0003_2011-24.xlsx


14100DO0003_2011-24.xlsx: 100%|██████████| 19.7M/19.7M [00:00<00:00, 20.5MB/s]


<h1>Data Integration and Cleaning Data </h1>

In [107]:
class cleaningData:
    def __init__(self):
        pass

    def show_null_data(self, data:pd.DataFrame):
        '''
            Showing Null Data for each column in a single dataframe

            Args:
                data: Dataframe to analyze

        '''
        null_count = data.isna().sum()
        null_percent = data.isna().sum() / len(data)
        null_percent = null_percent.apply(lambda x: f"{x:.1%}")
        print("Total Duplicated Count: ", data.duplicated().sum())
        results = pd.concat([null_count, null_percent], axis=1)
        results.columns = ['Null Total Count', 'Null Percentage']
        return results
    
    def split_categorical_numerical(self, data:pd.DataFrame):
        '''
            Spliting Categorical and Numerical Columns

            Args:
                data: Dataframe
        '''
        numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
        categorical_cols = data.select_dtypes(include=['object', 'category']).columns
        
        return numerical_cols, categorical_cols
    
    def replacing_value(self,df:pd.DataFrame, existing_value, new_value):
        df.replace(existing_value, new_value, inplace=True)
        return None

    def filling_null_value(self, df:pd.DataFrame, replacing_value):
        cat_imputer = SimpleImputer(strategy='constant', fill_value="Unknown")
        df = cat_imputer.fit_transform(df)

        return None

class connectDB:
    def __init__(self):
        self.connection =  duckdb.connect('my_database.duckdb') 
    
    def create_new_table(self, df:pd.DataFrame, table_name:str):
        '''
            Creating New Table Based on Dataframe Columns


            Args:
                1. df: Dataframe or that we want to convert into database
                2. table_name: the name of the table that we want to create in duck db
        '''
        try:
            con = self.connection
            sql_types = {
                'object': 'VARCHAR',
                'int64': 'INTEGER',
                'float64': 'DOUBLE',
                'bool': 'BOOLEAN',
                'datetime64[ns]': 'TIMESTAMP'
            }

            columns_sql = []
            for col_name, dtype in df.dtypes.items():
                sql_type = sql_types.get(str(dtype), 'VARCHAR')  # Default to VARCHAR
                col_sql = f'"{col_name}" {sql_type}'
                columns_sql.append(col_sql)

            columns_str = ",\n    ".join(columns_sql)
            create_table_sql = f"CREATE TABLE {table_name} (\n    {columns_str}\n);"
            con.execute(create_table_sql)
            con.close()
        except Exception as err:
            con.close()
            print(err)
        
        return None
    
    def insert_data(self, df:pd.DataFrame, table_name: str, column_map: dict):
        '''
        Inserting database different column name with the csv

        Args:
            df: Dataframe to insert
            table_name: Name of the existing DuckDB table
            column_map: Mapping from dataframe columns to table columns
        
        column_map = {'df_col1': 'table_colA', 'df_col2': 'table_colB'}
        '''
        try:
            con = self.connection

            # Prepare SELECT clause using mapping
            select_clause = ", ".join([f'"{df_col}" AS "{table_col}"' for df_col, table_col in column_map.items()])
            
            # Register DataFrame as a relation
            con.register("df_temp", df)
            
            # Build INSERT SQL
            sql = f"""
            INSERT INTO {table_name} ({', '.join(column_map.values())})
            SELECT {select_clause} FROM df_temp
            """
            con.execute(sql)
            con.close()
        except Exception as err:
            con.close()
            print(err)
        


        


In [109]:

# first url file
df_nger = pd.read_csv("NGER.ID0243.csv")

# second url file
df_cer_1_approved = pd.read_csv("power-stations-and-projects-accredited.csv")
df_cer_1_commited = pd.read_csv("power-stations-and-projects-committed.csv")
df_cer_1_probable = pd.read_csv("power-stations-and-projects-probable.csv")


# third url file
df_abs = pd.ExcelFile("14100DO0003_2011-24.xlsx")
statistical_area = pd.read_excel(df_abs, 'Table 1', header=[5,6])
lga_area = pd.read_excel(df_abs, 'Table 2', header=[5,6])





#### Cleaning the NGER.CSV dataset and put it into DB

In [ ]:
# module for Cleaning Data
cleaning_data = cleaningData()


df_nger_numerical, df_nger_categorical = cleaning_data.split_categorical_numerical(df_nger) 
cleaning_data.replacing_value(df_nger, '-', np.nan)

df_nger_cleaning = df_nger.drop(columns=['Electricity production GJ', 'Total emissions t CO2 e'])
df_nger_clean = cleaning_data.filling_null_value(df_nger_cleaning[df_nger_categorical], "Unknown")

df_nger_cleaning.describe()



,Electricity production MWh,Total scope 1 emissions t CO2 e,Total scope 2 emissions t CO2 e,Emission intensity t CO2 e MWh
count,7.750000e+02,7.750000e+02,775.000000,624.000000
mean,5.659881e+05,3.602111e+05,5419.553548,0.330769
std,2.308429e+06,2.104691e+06,43243.487455,0.504010
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,4.636000e+03,2.400000e+01,0.000000,0.000000
50%,7.232800e+04,3.080000e+02,29.000000,0.070000
75%,2.903670e+05,3.538500e+03,614.500000,0.660000
max,3.737227e+07,3.284074e+07,691029.000000,8.570000


In [ ]:
cleaning_data.replacing_value(df_cer_1_approved, '-', np.nan)

df_cer_1_approved['power_station_name_clean'] = df_cer_1_approved['Power station name'].str.split('-').str[0]
cleaning_data.show_null_data(df_cer_1_approved)


Total Duplicated Count:  0


,Null Total Count,Null Percentage
Accreditation code,0,0.0%
Power station name,0,0.0%
State,0,0.0%
Postcode,0,0.0%
Installed capacity (MW),0,0.0%
Fuel Source (s),0,0.0%
Accreditation start date,0,0.0%
Approval date,0,0.0%
power_station_name_clean,0,0.0%


In [ ]:
cleaning_data.replacing_value(df_cer_1_commited, '-', np.nan)

cleaning_data.show_null_data(df_cer_1_commited)

Total Duplicated Count:  0


,Null Total Count,Null Percentage
Project Name,0,0.0%
State,0,0.0%
MW Capacity,0,0.0%
Fuel Source,0,0.0%
Committed Date (Month/Year),0,0.0%


In [ ]:
cleaning_data.replacing_value(df_cer_1_probable, '-', np.nan)
cleaning_data.show_null_data(df_cer_1_probable)

Total Duplicated Count:  0


,Null Total Count,Null Percentage
Project Name,0,0.0%
State,0,0.0%
MW Capacity,0,0.0%
Fuel Source,0,0.0%


In [ ]:
category_question = defaultdict(list)
for cat, q in statistical_area.columns:
    category_question[cat].append(q)
category_question = dict(category_question)

In [ ]:
# only get the important rows
statistical_area = statistical_area.iloc[:-7]
lga_area = lga_area.iloc[:-7]

In [ ]:
cleaning_data.replacing_value(statistical_area, '-', np.nan)

statistical_area[("Unnamed: 2_level_0", "Year")] = pd.to_numeric(statistical_area[("Unnamed: 2_level_0", "Year")], errors="coerce").astype("Int64") 
statistical_area.head()

/var/folders/2h/b49chrcn56z4tf_5y0zw27740000gn/T/ipykernel_99065/3946964666.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(existing_value, new_value, inplace=True)


Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Code              Label               Year   
0                AUS          Australia               2011   
1                AUS          Australia               2016   
2                AUS          Australia               2017   
3                AUS          Australia               2018   
4                AUS          Australia               2019   

   Number of businesses - at 30 June  \
  Number of non-employing businesses   
0                                NaN   
1                                NaN   
2                                NaN   
3                                NaN   
4                                NaN   

                                                 \
  Number of employing businesses: 1-4 employees   
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   

                                                  \
  Number of employing businesses: 5-19 employees   
0                                            NaN   
1                                            NaN   
2                                            NaN   
3                                            NaN   
4                                            NaN   

                                                        \
  Number of employing businesses: 20 or more employees   
0                                                NaN     
1                                                NaN     
2                                                NaN     
3                                                NaN     
4                                                NaN     

                                Business entries - year ended 30 June  \
  Total number of businesses Number of non-employing business entries   
0                        NaN                                      NaN   
1                        NaN                                      NaN   
2                        NaN                                      NaN   
3                        NaN                                      NaN   
4                        NaN                                      NaN   

                                                       ...  \
  Number of employing business entries: 1-4 employees  ...   
0                                                NaN   ...   
1                                                NaN   ...   
2                                                NaN   ...   
3                                                NaN   ...   
4                                                NaN   ...   

  Estimated dwelling stock - as at June quarter                               \
                           Houses - total (no.) Townhouses - additions (no.)   
0                                           NaN                          NaN   
1                                           NaN                          NaN   
2                                     7279135.0                      31729.0   
3                                     7371408.0                      33346.0   
4                                     7472853.0                      32571.0   

                                                        \
  Townhouses - removals (no.) Townhouses - total (no.)   
0                         NaN                      NaN   
1                         NaN                      NaN   
2                       480.0                1261457.0   
3                       919.0                1293884.0   
4                       712.0                1325743.0   

                                                            \
  Apartments - additions (no.) Apartments - removals (no.)   
0                          NaN                         NaN   
1                          NaN                         

In [ ]:
cleaning_data.replacing_value(lga_area, '-', np.nan)

lga_area[("Unnamed: 2_level_0", "Year")] = pd.to_numeric(lga_area[("Unnamed: 2_level_0", "Year")], errors="coerce").astype("Int64") 
lga_area.head()

/var/folders/2h/b49chrcn56z4tf_5y0zw27740000gn/T/ipykernel_99065/3946964666.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(existing_value, new_value, inplace=True)


Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Code              Label               Year   
0              10050             Albury               2011   
1              10050             Albury               2016   
2              10050             Albury               2017   
3              10050             Albury               2018   
4              10050             Albury               2019   

   Number of businesses - at 30 June  \
  Number of non-employing businesses   
0                                NaN   
1                                NaN   
2                                NaN   
3                                NaN   
4                                NaN   

                                                 \
  Number of employing businesses: 1-4 employees   
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   

                                                  \
  Number of employing businesses: 5-19 employees   
0                                            NaN   
1                                            NaN   
2                                            NaN   
3                                            NaN   
4                                            NaN   

                                                        \
  Number of employing businesses: 20 or more employees   
0                                                NaN     
1                                                NaN     
2                                                NaN     
3                                                NaN     
4                                                NaN     

                                Business entries - year ended 30 June  \
  Total number of businesses Number of non-employing business entries   
0                        NaN                                      NaN   
1                        NaN                                      NaN   
2                        NaN                                      NaN   
3                        NaN                                      NaN   
4                        NaN                                      NaN   

                                                       ...  \
  Number of employing business entries: 1-4 employees  ...   
0                                                NaN   ...   
1                                                NaN   ...   
2                                                NaN   ...   
3                                                NaN   ...   
4                                                NaN   ...   

  Industry of employment - Persons aged 15 years and over - Census  \
                       Rental, hiring and real estate services (%)   
0                                                1.3                 
1                                                1.3                 
2                                                NaN                 
3                                                NaN                 
4                                                NaN                 

                                                       \
  Professional, scientific and technical services (%)   
0                                                4.5    
1                                                4.3    
2                                                NaN    
3                                                NaN    
4                                                NaN    

                                           \
  Administrative and support services (%)   
0                                     2.8   
1                                     3.0   
2                                     NaN   
3                                     NaN   
4                                     NaN   

  

### Data Augmentation

In [ ]:
class geoAPI:
    def __init__(self):
        pass

    def get(self, query):
        url = "https://nominatim.openstreetmap.org/search"
        params = {
            "q": query,
            "format": "json",
            "limit": 1
        }
        headers = {"User-Agent": "powerstation-geocoder"} 
        response = requests.get(url, params=params, headers=headers)
        
        if response.status_code == 200 and len(response.json()) > 0:
            result = response.json()[0]
            return float(result["lat"]), float(result["lon"])
        else:
            return None, None

In [ ]:
api = geoAPI()

api.get("Laura Johnson Home, Townview")

(None, None)

In [ ]:
url = "http://wiki.openstreetmap.org/wiki/API"
headers = {
    "Content-Type": "application/json"
}

response = requests.get(url)
data = response.json()
print(data)

KeyboardInterrupt: 

#### Process Creating Schema in Duck DB

In [ ]:
# module related for database
database = connectDB()

column_name_facility = ['id','reporting_entity','facility_name','type','state',
                        'electricity_production_mwh','scope_1_emission', 
                        'scope_2_emission','emission_intesity_mwh','grid_connected',
                        'grid','primary_fuel', 'important_notes']

column_names_power_stations = ['id', 'accreditation_code', 'power_station_name','state','postcode',
                               'capacity_mw','source', 'accreditation_start_date','approval_date','commited_date','status']

# Create an empty DataFrame with the specified columns

In [ ]:

print("----------")
df_cer_1_approved.replace('-', np.nan, inplace=True)

----------


In [ ]:
print("----------")
#show_null_data(df_cer_1_commited)

----------


In [ ]:
print("----------")
#show_null_data(df_cer_1_probable)



----------


In [ ]:
print("----------")
#show_null_data(df_cer_2)

----------


In [ ]:
#show_null_data(df_cer_3)

In [ ]:

print("----------")
#show_null_data(statistical_area)


----------


In [ ]:
print("----------")
#show_null_data(lga_area)

----------
